<p style="padding: 10px; border: 1px solid black;">
<img src="images/MLU-NEW-logo.png" alt="drawing" width="400"/> <br/>


# <a name="0">MLU Advanced Prompt Engineering for LLMs</a>
## <a name="0">Lab 4: Reason and Act (ReAct)</a>

This notebook introduces ReAct, a general paradigm that combines reasoning and acting with LLMs. ReAct prompts LLMs to generate verbal reasoning traces and actions for a task. This allows the system to perform dynamic reasoning to create, maintain, and adjust plans for acting while also enabling interaction to external environments (e.g., Wikipedia, code interpreters) to incorporate additional information into the reasoning. 

1. <a href="#1">Import libraries</a>
2. <a href="#2">Set up Bedrock for inference</a>
3. <a href="#3">Reason and Act (ReAct)</a>
    - <a href="#31">LangChain tools</a>
    - <a href="#32">LangChain agents</a>
4. <a href="#4">ReAct examples</a>
    - <a href="#41">Time awareness</a>
    - <a href="#42">Calculator and code interpreter</a>
    - <a href="#43">Knowledge-intensive tasks</a>
    - <a href="#44">Decision-making tasks</a>
    
    
Please work top to bottom of this notebook and don't skip sections as this could lead to error messages due to missing code.

---

<br/>
You will be presented with coding activities to check your knowledge and understanding throughout the notebook whenever you see the MLU robot:

<img style="display: block; margin-left: auto; margin-right: auto;" src="./images/activity.png" alt="Activity" width="125"/>

### <a name="1">1. Import libraries</a>
(<a href="#0">Go to top</a>)

Let's start by installing all required packages as specified in the `requirements.txt` file and importing several libraries.

In [2]:
!pip install -q -U pip --root-user-action=ignore
!pip3 install -q -r requirements.txt --root-user-action=ignore

In [3]:
import boto3
import re
import random
import numpy as np

from datetime import date, time

from langchain.llms.bedrock import Bedrock
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun, tool
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.agents import AgentType, Tool, load_tools, initialize_agent

from IPython.display import Markdown, display

### <a name="2">2. Set up Bedrock for inference</a>
(<a href="#0">Go to top</a>)

To get started, set up Bedrock and instantiate an active `bedrock-runtime` to query LLMs. The code below leverages [LangChain's Bedrock integration](https://python.langchain.com/docs/integrations/llms/bedrock).

<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px">
    <h4>A note on Bedrock API invocation</h4>

Amazon Bedrock is generally available for all AWS customers. The provided lab for this course currently invokes Bedrock's external endpoint, which has access to Anthropic and AI21 models.

For training and learning purposes, Amazon Bedrock is also available through an internal endpoint that allows accessing Amazon's Titan models. To use that instead, please follow the instructions in the <a href="https://w.amazon.com/bin/view/AmazonBedrock/Products/GetStarted/">Amazon Bedrock Get Started wiki page</a>.
</div>
</br>

In [4]:
KEYS_INFERENCE = {
    "anthropic": {
        "max_tokens": "max_tokens_to_sample",
        "stop_sequences": "stop_sequences",
    },
    "ai21": {"max_tokens": "maxTokens", "stop_sequences": "stopSequences"},
    "amazon": {"max_tokens": "maxTokenCount", "stop_sequences": "stopSequences"}
}


# Define the bedrock-runtime client that will be used for predictions
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

########################### Select the LLM ###############################

# bedrock_model_id = "ai21.j2-mid-v1"
# bedrock_model_id = "ai21.j2-ultra-v1"
bedrock_model_id = "anthropic.claude-instant-v1"
# bedrock_model_id = "anthropic.claude-v2"

##########################################################################

# Each model has a different set of inference parameters
model_provider = bedrock_model_id.split(".")[0]
keys = KEYS_INFERENCE[model_provider]

inference_modifier = {
    "temperature": 0.0,
    keys["max_tokens"]: 1024,
    keys["stop_sequences"]: [],
}

llm = Bedrock(
    model_id=bedrock_model_id, client=bedrock_runtime, model_kwargs=inference_modifier
)

Next, use Bedrock for inference to test everything works as expected. 

We will first run the example with Claude Instant. Since the Anthropic models have been trained with [special tokens](https://docs.anthropic.com/claude/docs/human-and-assistant-formatting) to indicate the roles of the user and the assistant, we wrap every input to the model with the appropriate format. 

In [5]:
# Example model prompting

claude = lambda st: f"\n\nHuman: {st}\n\nAssistant:"

greeting_prompt = "Hello, what's the meaning of life?"

display(Markdown("---"))
display(Markdown(llm(claude(greeting_prompt))))
display(Markdown("---"))

---

 I don't have a definitive answer to the meaning of life. As an AI, I was created by Anthropic to be helpful, harmless, and honest. Different people may have different views on what gives their life meaning.

---

### <a name="3">3. Reason and Act (ReAct)</a>
(<a href="#0">Go to top</a>)

ReAct was introduced in [ReAct: Synergizing Reasoning and Acting in Language Models](https://arxiv.org/abs/2210.03629), by Yao et al. (2022) to explore the use of LLMs to generate both reasoning traces and task-specific actions in an interleaved manner. **Reasoning traces**, elicited e.g. by CoT, help the model induce, track, and update action plans as well as handle exceptions. **Actions**, induced e.g. by action plan generation, allow it to interface with external sources, such as knowledge bases or environments, to gather additional information.

The ReAct method involves augmenting prompts with relational constraints, allowing the LM to better capture the desired context. The key steps are as follows:

- **Identify entities**: define the entities in the context that are relevant to the task.
- **Formulate relations**: determine the relationships between the identified entities. These relations can be explicit (e.g., "X is a type of Y") or implicit (e.g., "X and Y are often found together").
- **Augment prompts**: combine the identified entities and their relations into a structured prompt that guides the LM towards generating contextually relevant response.

In general, ReAct outperforms CoT on question answering, fact verification, and decision making tasks. CoT might suffer from fact hallucination, however it is also true that ReAct's structural constraints reduce its flexibility in formulating reasoning steps. Also ReAct depends a lot on the information it's retrieving; non-informative search results might derail the model reasoning and lead to difficulty in recovering and reformulating thoughts.

Prompting methods that **combine and support switching between ReAct and CoT+Self-Consistency generally outperform** all the other approaches.

Below you will see an example of a Bedrock-hosted LLM improving its capabilities with ReAct via LangChain's built-in functionality that builds agents to perform tasks by combining the power of LLMs with different tools. 

#### <a name="31">3.1. LangChain tools</a>
(<a href="#0">Go to top</a>)

ReAct can be enhanced with specialized agents and tools. These can help in automating the process of identifying entities and relations, as well as in formulating structured prompts. 

Tools are functions that agents can use to interact with the world. These tools can be generic utilities (e.g. search), other chains, or even other agents. With tools, **LLMs can search the web, do math, run code**, and more. While the LangChain library provides a [substantial selection of pre-built tools](https://python.langchain.com/docs/integrations/tools/), in many real-world projects, we'll often find that existing tool might not work out-of-the-boxi as expected. Eventually we might need to modify existing tools or build entirely new ones.

#### Custom Tools

Let's start by building a custom tool to give models access to the current date, a piece of information that is very relevant to answer queries about the real world, and that LLMs do not have access to, as they lack awareness of present vs. past time.

The tool below returns the current date regardless of the input. Note that we enter the purpose of the tool in its `description`.

In [6]:
@tool
def current_date(text: str) -> str:
    """Returns today's date, use this for any questions related to knowing today's date. \
    When you use this function, the Action should be current_date. The Action Input should be "" \
    This function will always return today's date. \
    Any date math should occur outside this function."""
    return str(date.today())

# Define the date tool
date_tool = Tool(
    name="DateTool", func=current_date, description="Useful to retrieve the current date."
)

# Test date tool
display(Markdown("---"))
display(Markdown((date_tool("Whatever I input, the output should be today's date."))))
display(Markdown("---"))

---

2024-01-30

---

#### Python interpreter

LangChain supports the execution of Python code via the [PythonREPLTool](https://python.langchain.com/docs/integrations/toolkits/python) a tool to access Python's standard shell, or REPL (Read-Eval-Print Loop).

<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px">
    <h4>A note on security best practices with LLMs</h4>

The Python REPL tool is shown here as an example of what's possible to build with ReAct.
<br/>
This demo does not use or teach security best practices. You should not allow generative AI to run arbitrary code on production systems.</div>


#### Web search Tool

The tool below uses the `DuckDuckGo` search module to build a tool that allows to run web searches. Other APIs can be used to search on the web, however they might require setting up APIs and other resources, so we will leverage [DuckDuckGo](https://duckduckgo.com/) in this example.

In [7]:
# Define the API wrapper for DuckDuckGo search
duckduckgo_search_run = DuckDuckGoSearchRun()

# Define the DuckDuckGo tool using a description and the function to retrieve results from DuckDuckGo search
duckduckgo_general_tool = Tool(
    name="DuckDuckGoSearch General",
    func=duckduckgo_search_run.run,
    description="Useful for when you need to search online about facts and events or retrieve news"
)

# Test the DuckDuckGo tool
display(Markdown("---"))
display(Markdown(duckduckgo_general_tool("Taylor Swift")))
display(Markdown("---"))

---

/opt/conda/lib/python3.10/site-packages/langchain/utilities/duckduckgo_search.py:43: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


The official Taylor Swift reputation Stadium Tour tee is available now! Skip the long lines at the venues and show now HERE . Learn about the life and career of Taylor Swift, one of the biggest pop stars and 12-time Grammy winners. From country music to pop music, from awards to controversies, from relationships to activism, discover how she became a pop megastar and a pop culture icon. December 6, 2023 7:38 AM EST. T aylor Swift is telling me a story, and when Taylor Swift tells you a story, you listen, because you know it's going to be good—not only because she's had an ... And by Taylor Swift, we mean her boyfriend, Kansas City Chiefs tight end Travis Kelce, who led his team to victory against the Baltimore Ravens on Sunday night and clinched a spot in Feb. 11's ... David Letterman Says Taylor Swift Is a "Glowing Bright Light of Goodness". The former talk show host also expressed his support for Swift and Travis Kelce's relationship. By Christy Piña ...

---

#### Wikipedia Tool

The Wikipedia tool allows to retrieve information about people, places, companies, facts, historical events, and more.

In [8]:
from langchain.utilities import WikipediaAPIWrapper

# Define the API wrapper for Wikipedia
wikipedia = WikipediaAPIWrapper()

# Define the Wikipedia tool with a description and the function to retrieve results from Wikipedia
wikipedia_tool = Tool(
    name="wikipedia",
    func=wikipedia.run,
    description="Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.",
)

# Test the defined Wikipedia tool
display(Markdown("---"))
display(Markdown(wikipedia_tool("Who invented the airplane?")))
display(Markdown("---"))

---

Page: Airplane
Summary: An airplane (American English) or aeroplane (Commonwealth English), informally plane, is a fixed-wing aircraft that is propelled forward by thrust from a jet engine, propeller, or rocket engine. Airplanes come in a variety of sizes, shapes, and wing configurations. The broad spectrum of uses for airplanes includes recreation, transportation of goods and people, military, and research. Worldwide, commercial aviation transports more than four billion passengers annually on airliners and transports more than 200 billion tonne-kilometers of cargo annually, which is less than 1% of the world's cargo movement. Most airplanes are flown by a pilot on board the aircraft, but some are designed to be remotely or computer-controlled such as drones.
The Wright brothers invented and flew the first airplane in 1903, recognized as "the first sustained and controlled heavier-than-air powered flight". They built on the works of George Cayley dating from 1799, when he set forth the concept of the modern airplane (and later built and flew models and successful passenger-carrying gliders) and the work of German pioneer of human aviation Otto Lilienthal, who, between 1867 and 1896, also studied heavier-than-air flight. Lilienthal's flight attempts in 1891 are seen as the beginning of human flight.
Following its limited use in World War I, aircraft technology continued to develop. Airplanes had a presence in all the major battles of World War II. The first jet aircraft was the German Heinkel He 178 in 1939. The first jet airliner, the de Havilland Comet, was introduced in 1952. The Boeing 707, the first widely successful commercial jet, was in commercial service for more than 50 years, from 1958 to at least 2013.



Page: Shiva Ayyadurai
Summary: V. A. Shiva Ayyadurai (born Vellayappa Ayyadurai Shiva on December 2, 1963) is an Indian-American engineer, politician, entrepreneur, and anti-vaccine activist. He has become known for promoting conspiracy theories, pseudoscience, and unfounded medical claims. Ayyadurai holds four degrees from the Massachusetts Institute of Technology (MIT), including a PhD in biological engineering, and is a Fulbright grant recipient.In a 2011 article published by Time, Ayyadurai claimed to have invented email, as a teenager; in August 1982, he registered the copyright on an email application he had written. Historians strongly dispute this account because email was already in use in the early 1970s. Ayyadurai sued Gawker Media and Techdirt for defamation for disputing his account of inventing email; both lawsuits were settled out of court. Ayyadurai and Techdirt agreed to Techdirt's articles remaining online with a link to Ayyadurai's rebuttal on his own website. Ayyadurai also attracted attention for two reports: the first questioning the working conditions of India's largest scientific agency; the second questioning the safety of genetically modified food, such as soybeans. During the COVID-19 pandemic, Ayyadurai became known for a social media COVID-19 disinformation campaign, spreading conspiracy theories about the cause of COVID-19, promoting unfounded COVID-19 treatments, and campaigning to fire Anthony Fauci for allegedly being a deep state actor.
Ayyadurai garnered 3.39% of the vote as an independent candidate in the 2018 U.S. Senate election in Massachusetts, and ran for the Republican Party in the 2020 U.S. Senate election in Massachusetts but lost to Kevin O'Connor in the primary. After the election, he promoted claims of election fraud that were shown to be false by fact checkers.



Page: List of inventors killed by their own invention
Summary: This is a list of people whose deaths were in some manner caused by or directly related to a product, process, procedure, or other technological innovation that they invented or designed.



---


<div style="border: 4px solid coral; text-align: center; margin: auto; padding-left: 50px; padding-right: 50px">
    <h2><i>Try it Yourself!</i></h2>
    <p style="text-align:center;margin:auto;"><img src="./images/activity.png" alt="Activity" width="100" /> </p>
    <p style=" text-align: center; margin: auto;"><b>Try creating more tools for other tasks.</b></br> For example, you can ask use <a href="https://python.langchain.com/docs/integrations/tools/yahoo_finance_news">YahooFinanceNewsTool</a> to query for financial information from companies such as <code>AMZN</code>.
</p>
    <br>
</div>

#### <a name="32">3.2. LangChain agents</a>
(<a href="#0">Go to top</a>)

An Agent is a wrapper around a model, which takes in user input and returns a response corresponding to an "action" to take and a corresponding "action input". With agents, an **LLM is used as a reasoning engine** to determine which actions to take and in which order.

LangChain provides a utility to initialize an agent. To do so, we need to pass the following parameters:

- **`tools`**: a list of tools that the model has access to
- **`llm`**: the language model that will perform the reasoning and act process
- **`agent`**: the type of agent that we want to initialize

The example below initializes a "Zero-shot React Description" agent. Zero-shot means the agent functions on the current action only, i.e. it has no memory. The agent uses the ReAct framework to decide which tool to use, based solely on the tool's `description`. It is important to give the agent access to a correct set of tools, otherwise it will not be able to accomplish its objective. Additionally, if tools are not properly described, the agent won't know how to use them.

The code below initializes an agent with the following capabilites:

- identify current date
- access to Python interpreter
- web search capabilities (general, trips, hotels, flights)
- wikipedia queries


In [9]:
tools = [
    current_date,
    PythonREPLTool(),
    duckduckgo_general_tool,
    wikipedia_tool
]

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

### Customizing the agent's prompt template

LangChain's Zero-shot React Description agent orchestrates the sequence of thoughts-actions-observations via a default template that is stored in `agent.agent.llm_chain.prompt.template` as you can see below. 

For more control on the ReAct results, it is possible to **customize the LLM agent's prompt template**, as demonstrated in this [Custom LLM agent walkthrough](https://python.langchain.com/docs/modules/agents/how_to/custom_llm_agent). This way, one can force the agent to output the final answer in a special format or style. Similarly, one can make rules to the LLM explicit in the prompt template, for instance adding an instruction for the LLM to always use a particular tool as the first tool or enforcing a specific order constraint when calling the available tools. 

In [10]:
# Default prompt template of AgentType.ZERO_SHOT_REACT_DESCRIPTION
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

current_date: current_date(text: str) -> str - Returns today's date, use this for any questions related to knowing today's date.     When you use this function, the Action should be current_date. The Action Input should be ""     This function will always return today's date.     Any date math should occur outside this function.
Python_REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.
DuckDuckGoSearch General: Useful for when you need to search online about facts and events or retrieve news
wikipedia: Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think a

#### Util function to run ReAct queries

The util function below will be used to run queries with and without ReAct for easy comparison of the model's performance.

In [11]:
def run_standard_and_react(llm, agent, prompt):
    """
    Utility function to prompt an LLM with a standard query and with ReAct through an agent 
    """
    display(Markdown("---"))
    display(Markdown(prompt))
    display(Markdown("---"))
    display(Markdown("### Without ReAct"))
    display(Markdown(llm(prompt)))
    display(Markdown("---"))
    display(Markdown("### With ReAct"))
    display(Markdown(agent.run(prompt)))
    display(Markdown("---"))

### <a name="4">4. ReAct examples</a>
(<a href="#0">Go to top</a>)

It is time to put our agent to work. Let's create an agent with current time and Python capabilities.

#### <a name="41">4.1. Time awareness</a>
(<a href="#0">Go to top</a>)

The LLM can gain awareness of the current time, since it has access to our custom date tool. Compare the results obtained with the LLM as a ReAct agent versus standard prompting without ReAct:

<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px">
    <h4>A note on security best practices with LLMs</h4>

The Python REPL tool is used here as an example of what's possible to do with ReAct.
<br/>
This demo does not use or teach security best practices. You should not allow generative AI to run arbitrary code on production systems.</div>


In [12]:
prompt = claude("I was born on March 22, 2000. How old am I today in years, months and days?")
run_standard_and_react(llm, agent, prompt)

---



Human: I was born on March 22, 2000. How old am I today in years, months and days?

Assistant:

---

### Without ReAct

 Okay, here are the steps:

1) You were born on March 22, 2000
2) The current date is May 25, 2022
3) To calculate your age in years: 2022 - 2000 = 22 years
4) To calculate your age in months: Take the number of full years (22) and multiply by 12 months/year. Then add the number of months since your last birthday. 22 * 12 = 264 months. Since your last birthday was in March, and it's now May, that's 2 additional months. So 264 + 2 = 266 months.  
5) To calculate your age in days: Take the number of full years (22) and multiply by 365 days/year. Then add the number of days since your last birthday. 22 * 365 = 8030 days. Since your last birthday was March 22, and it's now May 25, that's 64 additional days. So 8030 + 64 = 8094 days.

Therefore, your current age is:
22 years
266 months 
8094 days

...



> Entering new AgentExecutor chain...
 I need to know today's date to calculate the age
Action: current_date
Action Input: ""
Observation: 2024-01-30
Thought:

Python REPL can execute arbitrary code. Use with caution.


 I can use Python to calculate the difference between the birthdate and today's date
Action: Python_REPL
Action Input: import datetime
from datetime import date

birthdate = date(2000, 3, 22)
today = date(2024, 1, 30)
age = today - birthdate
print(age)

Observation: 8714 days, 0:00:00

Thought: I need to convert the days to years, months and days
Action: Python_REPL 
Action Input: import datetime

from datetime import date

birthdate = date(2000, 3, 22)
today = date(2024, 1, 30)
age_in_days = (today - birthdate).days
age = today.year - birthdate.year - 1
if today < date(today.year, birthdate.month, birthdate.day):
    age += 1

print(f"{age} years, {age_in_days//365} months, {age_in_days%365} days")

Observation: 24 years, 23 months, 319 days

Thought: I now have the age in years, months and days
Final Answer: 24 years, 23 months, 319 days

> Finished chain.


24 years, 23 months, 319 days

---

#### <a name="42">4.2. Calculator and Code interpreter</a>
(<a href="#0">Go to top</a>)

The ability to run Python code greatly enhances the performance of LLMs in some arithmetic as well as logical tasks with a clear algorithmic solution.

In [13]:
# EXAMPLE 1
# Arithmetic calculation that an LLM might get wrong

prompt = claude("What's the square root of 2.5 million?")
run_standard_and_react(llm, agent, prompt)

---



Human: What's the square root of 2.5 million?

Assistant:

---

### Without ReAct

 Here are the step-by-step workings:
1) 2.5 million = 2,500,000
2) Take the square root of 2,500,000
3) The square root of 2,500,000 is 5000

Therefore, the square root of 2.5 million is 5000.

---

### With ReAct



> Entering new AgentExecutor chain...
 To find the square root of a number, I need to use the math module in Python.
Action: Python_REPL
Action Input: import math; print(math.sqrt(2500000))
Observation: 1581.1388300841897

Thought: I now have the numerical value of the square root of 2.5 million.
Final Answer: The square root of 2.5 million is approximately 1581.14.

> Finished chain.


The square root of 2.5 million is approximately 1581.14.

---

In [12]:
# EXAMPLE 2
# Sorting of words that an LLM migth get wrong

prompt = claude("Invert the letters of the following words and sort them in reverse order: apple, Banana, Cherry, pear, elderberry")
run_standard_and_react(llm, agent, prompt)

---



Human: Invert the letters of the following words and sort them in reverse order: apple, Banana, Cherry, pear, elderberry

Assistant:

---

### Without ReAct

 Here are the words with inverted letters sorted in reverse order:

elderberry => yrerbdel
pear => raep
Cherry => yrrehC
Banana => ananaB 
apple => elppa

---

### With ReAct



> Entering new AgentExecutor chain...
 I need to invert the letters of each word and then sort them in reverse alphabetical order
Action: Python_REPL
Action Input: words = ["apple", "Banana", "Cherry", "pear", "elderberry"]
inverted = [word[::-1] for word in words]
print(sorted(inverted, reverse=True))
Observation: ['yrrehC', 'yrrebredle', 'raep', 'elppa', 'ananaB']

Thought: The final answer is the output of running the Python code to invert and sort the letters of the words in reverse order.
Final Answer: ['yrrehC', 'yrrebredle', 'raep', 'elppa', 'ananaB']

> Finished chain.


['yrrehC', 'yrrebredle', 'raep', 'elppa', 'ananaB']

---

---

<div style="border: 4px solid coral; text-align: center; margin: auto; padding-left: 50px; padding-right: 50px">
    <h2><i>Try it Yourself!</i></h2>
    <p style="text-align:center;margin:auto;"><img src="./images/activity.png" alt="Activity" width="100" /> </p>
    <p style=" text-align: center; margin: auto;"><b>Try solving similar tasks involving math and code.</b></br> For example, you can ask the model to compute compound interest over the years, or to answer arithmetic or trigonometric math problems.</code>
</p>
    <br>
</div>

In [13]:
############## CODE HERE ####################



############## END OF CODE ##################

#### <a name="43">4.3. Knowledge-intensive tasks</a>
(<a href="#0">Go to top</a>)

We will now use an agent enhanced with web search and Wikipedia query capabilites and probe it in its ability to answer questions about facts of the real world. The example below requires certain knowledge about persons and facts.

In [14]:
prompt = claude("Which AWS service provides access to foundation models?")
run_standard_and_react(llm, agent, prompt)

---



Human: Which AWS service provides access to foundation models?

Assistant:

---

### Without ReAct

 Amazon Web Services (AWS) provides access to foundation models through Amazon Lex, an AI assistant creation service. With Lex, you can build conversational interfaces into any application using voice and text. Lex uses advanced deep learning technologies like deep neural networks and natural language understanding to enable conversations between humans and bots.

Some key points:

- Amazon Lex allows developers to build conversational bots or digital assistants with natural language understanding capabilities powered by deep learning technologies.

- Lex bots can understand what users say (speech recognition), determine what they mean (natural language understanding), and respond in a natural way (natural language generation).

- Under the hood, Lex bots leverage large language models and deep neural networks trained on massive amounts of text, similar to other foundation models.

- Developers can build Lex bots through an easy-to-use web interface without having to build, train or deploy their own deep learning models.

- Once built, Lex bots can be integrated into applications, devices, and services to enable natural conversations via voice, text, or both.

So in summary, the AWS service that provides access to foundation model technologies for building conversational interfaces is Amazon Lex. It handles all the deep learning underpinnings and allows developers to focus on the conversational aspects.

---

### With ReAct



> Entering new AgentExecutor chain...
 I need to search online for AWS services
Action: DuckDuckGoSearch General
Action Input: AWS services for foundation models
Observation: Amazon Bedrock is a fully managed service that offers a choice of high-performing foundation models (FMs) from leading AI companies, including AI21 Labs, Anthropic, Cohere, Stability AI, and Amazon, along with a broad set of capabilities to build generative AI applications, simplifying the development while maintaining privacy and security. This April, Swami Sivasubramanian, Vice President of Data and Machine Learning at AWS, announced Amazon Bedrock and Amazon Titan models as part of new tools for building with generative AI on AWS. Amazon Bedrock, currently available in preview, is a fully managed service that makes foundation models (FMs) from Amazon and leading AI startups—such as […] Amazon Bedrock is a new service for building and scaling generative AI applications, which are applications that can generate

Amazon Bedrock

---

<div style="border: 4px solid coral; text-align: center; margin: auto; padding-left: 50px; padding-right: 50px">
    <h2><i>Try it Yourself!</i></h2>
    <p style="text-align:center;margin:auto;"><img src="./images/activity.png" alt="Activity" width="100" /> </p>
    <p style=" text-align: center; margin: auto;"><b>Try solving similar knowledge-based tasks.</b></br> For example, you can ask the model to tell you facts about famous actors, scientists, singers, or historical characters.</code>
</p>
    <br>
</div>

In [15]:
############## CODE HERE ####################



############## END OF CODE ##################

#### <a name="44">4.4. Decision-making tasks</a>
(<a href="#0">Go to top</a>)

We will now leverage the planning capabilities of the agent enhanced with custom web search to ask the model to put together complex plans involving a set of decision. The example below is about deciding where to go on an upcoming impromptu vacation.

In [16]:
instruction = """Help me make a one-week plan for a last-minute vacation for next week.
I live in Munich and I'm considering either Tokyo or Las Vegas.
Once you've chosen the best destination, produce a plan with activities for each day."""

prompt = claude(instruction)
run_standard_and_react(llm, agent, prompt)

---



Human: Help me make a one-week plan for a last-minute vacation for next week.
I live in Munich and I'm considering either Tokyo or Las Vegas.
Once you've chosen the best destination, produce a plan with activities for each day.

Assistant:

---

### Without ReAct

 Here is a suggested one-week itinerary for a last-minute vacation to Tokyo:

Day 1: 
- Fly to Tokyo (flight time approx. 11 hours)
- Check into hotel, rest and recover from jet lag

Day 2: 
- Visit Meiji Jingu shrine and walk around Harajuku for shopping and street fashion
- Wander through Shibuya and see the famous Shibuya crossing

Day 3: 
- Visit Senso-ji temple in Asakusa and explore Nakamise-dori shopping street
- Stroll around Ueno Park and visit the Tokyo National Museum

Day 4:
- Visit the Imperial Palace gardens and Plaza 
- Walk around Ginza and do some high-end shopping

Day 5: 
- Visit Tokyo Skytree and enjoy views of the city from the observation decks
- Explore Akihabara, the electric town, and check out anime and gaming stores 

Day 6: 
- Visit Tsukiji Fish Market and have sushi for breakfast 
- Relax in Hamarikyu Gardens by the water

Day 7:
- Visit one of the many onsen hot spring resorts outside Tokyo for relaxation (e.g. Hakone)
- Fly back from Tokyo in the evening

I chose Tokyo over Las Vegas as it offers much more cultural activities and sights to experience in a week. The plan includes a good mix of major attractions, parks, temples, and neighborhoods to explore Tokyo's unique culture, history, and energy. Let me know if you need any other recommendations!

---

### With ReAct



> Entering new AgentExecutor chain...
 I need to consider factors like travel time, cost, and things to do in each location to pick the best destination.
Action: DuckDuckGoSearch General
Action Input: travel time munich to tokyo vs las vegas
Observation: Munich is 7 hours behind of Tokyo. If you are in Munich, the most convenient time to accommodate all parties is between 9:00 am and 11:00 am for a conference call or meeting. In Tokyo, this will be a usual working time of between 4:00 pm and 6:00 pm. If you want to reach out to someone in Tokyo and you are available anytime, you can schedule a ... Las Vegas is 16 hours behind of Tokyo. If you are in Las Vegas, the most convenient time to accommodate all parties is between 5:00 pm and 6:00 pm for a conference call or meeting. In Tokyo, this will be a usual working time of between 9:00 am and 10:00 am. If you want to reach out to someone in Tokyo and you are available anytime, you can ... Pack clothing for all kinds of weather. The wea

Based on the information researched, I would recommend Tokyo as the destination for the one-week last-minute vacation next week. The plan outlined above provides a good mix of major sightseeing spots and activities for each day, fully utilizing the time available and showcasing different aspects of Tokyo. While Las Vegas would be shorter travel time, Tokyo offers much more cultural immersion and things to see and do over the course of a week.

---

<div style="border: 4px solid coral; text-align: center; margin: auto; padding-left: 50px; padding-right: 50px">
    <h2><i>Try it Yourself!</i></h2>
    <p style="text-align:center;margin:auto;"><img src="./images/activity.png" alt="Activity" width="100" /> </p>
    <p style=" text-align: center; margin: auto;"><b>Try solving similar planning tasks.</b><br/>For example, you can ask the model to plan a party and make a shopping list to be ordered in Amazon.</p>
    <br>
</div>

In [17]:
############## CODE HERE ####################



############## END OF CODE ##################

### Conclusion

We have examined a series of tasks that are typically challenging for LLMs to solve, as they involve time awareness, compute capabilities, factual knowledge about current and past persons and events, and the ability to reason and make decisions. The ReAct approach allows the model to generate verbal reasoning traces and actions for a task. This helps make LLM responses more explainable and reduces hallucinations compared to CoT, giving users a better chance of solving the tasks they want.

It is important to keep in mind that current agents relies on natural language as an interface between LLMs and additional tools. However, the **reliability of model outputs with ReAct is questionable**, as LLMs may make formatting errors and occasionally exhibit rebellious behavior (e.g. refuse to follow an instruction). Consequently, a great deal of work needs to be invested in crafting appropriate prompts and in parsing model output correctly.

<div style="border: 4px solid coral; text-align: center; margin: auto; padding-left: 50px; padding-right: 50px">
    <h2><i>Try it Yourself!</i></h2>
    <br>
    <p style="text-align:center;margin:auto;"><img src="./images/activity.png" alt="Activity" width="100" /> </p>
    <p style=" text-align: center; margin: auto;"><b>Well done on completing the lab. Now it's time for you to get creative.</b></p>
    <p style=" text-align: center; margin: auto;">Try getting the model to solve other tasks for you.</p>
    <p style=" text-align: center; margin: auto;">You can for instance incorporate finance questions using  <code>YahooFinanceNewsTool</code> or medical informating using the <code>PubmedQueryRun</code> as tools.</p>
    <br>
</div>


In [18]:
############## CODE HERE ####################



############## END OF CODE ##################

# Thank you!

<p style="padding: 10px; border: 1px solid black;">
<img src="images/MLU-NEW-logo.png" alt="drawing" width="400"/> <br/>